In [1]:
import pandas as pd
import numpy as np
from utils import get_species, get_labels, get_labels_all

In [2]:
X, y, y_all = get_species(), get_labels(), get_labels_all()

In [3]:
X = X[list(filter(lambda x : "virus" not in x and "unclassified" not in x, X.columns))]

In [4]:
weights = ((y_all == "T2D") | (y_all == "Obesity") | (y_all == "Overweight") | (y_all == "Underweight")).values
weights

array([[False],
       [False],
       [False],
       ...,
       [ True],
       [ True],
       [ True]])

In [11]:
seq = pd.read_csv("../data/seq.csv", index_col=[0, 1])
seq

Sequencing Platform
Study_ID  Sample Accession                     
GMHI-23   SAMEA3879547      Illumina HiSeq 2000
          SAMEA3879551      Illumina HiSeq 2000
          SAMEA3879543      Illumina HiSeq 2000
          SAMEA3879565      Illumina HiSeq 2000
          SAMEA3879546      Illumina HiSeq 2000
...                                         ...
GMHI-17   SAMEA2737871      Illumina HiSeq 2000
          SAMEA2737819      Illumina HiSeq 2000
          SAMEA2737851      Illumina HiSeq 2000
GMHI-V-38 SAMD00114974      Illumina HiSeq 2500
GMHI-3    SAMEA2466888      Illumina HiSeq 2000

[12532 rows x 1 columns]

In [12]:
X_red, y_red, y_all_red, seq_red = X.iloc[~weights, :], y.iloc[~weights, :], y_all.iloc[~weights, :], seq.iloc[~weights, :]
# X_red, y_red, y_all_red = X, y, y_all

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, seq_train, seq_test = train_test_split(X_red, y_red, seq_red, test_size=0.2, random_state=42)

In [14]:
# np.random.seed(42)

# studies = np.unique(X_red.index.get_level_values(0))
# shuffle = np.random.permutation(np.arange(len(studies)))
# test = studies[shuffle[:len(studies) // 5]]
# train = studies[shuffle[len(studies) // 5:]]

# X_train, X_test, y_train, y_test = X_red.loc[train], X_red.loc[test], y_red.loc[train], y_red.loc[test]

In [15]:
from sklearn.preprocessing import MinMaxScaler

c = 0.00001
# minmax = MinMaxScaler()
# X_train_scaled = minmax.fit_transform(np.log(X_train + c))
# X_test_scaled = minmax.transform(np.log(X_test + c))
X_train_scaled = X_train > c
X_test_scaled = X_test > c

In [28]:
from sklearn.preprocessing import OneHotEncoder

oh = OneHotEncoder(sparse=False)

X_train_scaled2 = np.hstack([X_train_scaled, oh.fit_transform(seq_train)])
X_test_scaled2 = np.hstack([X_test_scaled, oh.transform(seq_test)])
X_train_scaled2.shape

(7728, 1407)

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score

clf = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=1, class_weight="balanced")
clf.fit(X_train_scaled, y_train)
y_hat = clf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_hat), accuracy_score(y_test, y_hat)

/Users/daniel/opt/anaconda3/envs/xgboost/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(0.7726714901207119, 0.7718572167615106)

In [79]:
(np.abs(clf.coef_) > 0).sum()

595

In [172]:
from sklearn.svm import SVC

clf = SVC(random_state=42, C=1, class_weight="balanced")
clf.fit(X_train > c, y_train)
y_hat = clf.predict(X_test > c)
balanced_accuracy_score(y_test, y_hat), accuracy_score(y_test, y_hat)

(0.7771576792151839, 0.7806518365235385)

In [89]:
from GMHI import GMHI
best = -1
for theta_f in [1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2]:
    for theta_d in [0.05, 0.1, 0.15, 0.2]:
        g = GMHI(use_shannon=False, theta_f=theta_f, theta_d=theta_d)
        g.fit(X_train, y_train)
        y_hat = g.predict(X_test)
        best = max(best, balanced_accuracy_score(y_test, y_hat))
best

0.7191292660561983